In [12]:
import cv2
import matplotlib.pyplot as plt
from ipywidgets import interact
import numpy as np
from scipy import stats
from common.utils import scan_directory
import os

ModuleNotFoundError: No module named 'dataset'

In [16]:
def ImageLoad(image, echo='off'):
    if echo == 'on':
        print('Loading image ' + image + '...')

    img = cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB)
    
    # Получение текущих размеров изображения
    height, width = img.shape[:2]
    
    # Вычисление новых размеров, сохраняя пропорции
    new_width = 512
    new_height = int(height * (new_width / width))
    
    # Масштабирование изображения до новых размеров
    resized_img = cv2.resize(img, (new_width, new_height))

    return resized_img
    
feature_extractor = cv2.SIFT_create()

t = 0
Xt = np.zeros((10000000,128))
yt = np.zeros((10000000,))

images_path = scan_directory(r"C:\Users\UserPC\visual_comparison\visdatcompy\dataset")

images_fpath = list(map(lambda x: os.path.join(x[0], x[1]), images_path))

N = len(images_fpath)
print (images_fpath)
for i in range(N):
  I = ImageLoad(images_fpath[i], echo='on')
  J = cv2.cvtColor(I, cv2.COLOR_RGB2GRAY)
  # SIFT extraction
  kp, desc = feature_extractor.detectAndCompute(J, None)
  ni = desc.shape[0]
  for j in range(ni):
    f = desc[j,:]
    Xt[t,:] =  f/np.linalg.norm(f)
    yt[t] = i
    t = t+1

  print('... '+str(ni)+' Дескрипторы были извлечены.')

X = Xt[0:t,:]
y = yt[0:t]
print('Количество дескрипторов SIFT в '+str(N)+' images: '+str(t))

['C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\000.jpg', 'C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\001.jpg', 'C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\002.jpg', 'C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\003.jpg', 'C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\004.jpg', 'C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\005.jpg', 'C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\006.jpg', 'C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\008.jpg', 'C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\011.jpg', 'C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\013.jpg', 'C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\014.jpg', 'C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\015.jpg', 'C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\016.jpg', 'C:\\Users\\UserPC\\visual_comparison\\visdatcompy\\dataset\\018.jpg', 'C:\\

... 552 Дескрипторы были извлечены.
Loading image C:\Users\UserPC\visual_comparison\visdatcompy\dataset\001.jpg...
... 713 Дескрипторы были извлечены.
Loading image C:\Users\UserPC\visual_comparison\visdatcompy\dataset\002.jpg...
... 767 Дескрипторы были извлечены.
Loading image C:\Users\UserPC\visual_comparison\visdatcompy\dataset\003.jpg...
... 1230 Дескрипторы были извлечены.
Loading image C:\Users\UserPC\visual_comparison\visdatcompy\dataset\004.jpg...
... 609 Дескрипторы были извлечены.
Loading image C:\Users\UserPC\visual_comparison\visdatcompy\dataset\005.jpg...
... 1188 Дескрипторы были извлечены.
Loading image C:\Users\UserPC\visual_comparison\visdatcompy\dataset\006.jpg...
... 1163 Дескрипторы были извлечены.
Loading image C:\Users\UserPC\visual_comparison\visdatcompy\dataset\008.jpg...
... 1360 Дескрипторы были извлечены.
Loading image C:\Users\UserPC\visual_comparison\visdatcompy\dataset\011.jpg...
... 351 Дескрипторы были извлечены.
Loading image C:\Users\UserPC\visual_com

In [18]:
def find_similar_image(itest, X, y):
    ik = itest
    ii = np.where(y==ik)[0]  
    jj = np.where(y!=ik)[0]  

    Xi = X[ii,:]  
    Xj = X[jj,:]  
    yj = y[jj]    

    Dt = np.dot(Xj,Xi.T)

    n = Xi.shape[0]

    z = np.zeros((n,))
    d = np.zeros((n,))

    for k in range(n):
        h    = Dt[:,k]         
        i    = h.max()         
        z[k] = i
        j    = np.where(h==i)  
        d[k] = yj[j]           

    kk = np.where(z>0.9)

    m = stats.mode(d[kk])
    ifound = int(m[0]) 

    return ifound

def visualize_similar_images(itest):
    ifound = find_similar_image(itest, X, y)
    I = ImageLoad(images_fpath[itest],echo='on')
    print('Тестовое изображение: '+str(itest))
    plt.imshow(I,cmap='gray')
    plt.axis('off')
    plt.show()
    print(' ')
    J = ImageLoad(images_fpath[ifound],echo='on')
    print('Для изображения '+str(itest)+', наиболее похожее изображение: '+str(ifound)+'.')
    plt.imshow(J,cmap='gray')
    plt.axis('off')
    plt.show()
    print('--------------------------------------------------')

# Пример использования с ipywidgets interact
interact(visualize_similar_images, itest=(0, 400))


interactive(children=(IntSlider(value=200, description='itest', max=400), Output()), _dom_classes=('widget-int…

<function __main__.visualize_similar_images(itest)>